# Example: Registered Interest and Principal of Securities (STRIPS) Bonds

## Background
[Registered Interest and Principal of Securities (STRIPS) bonds](https://en.wikipedia.org/wiki/United_States_Treasury_security#STRIPS) are a unique type of fixed-income investment instrument that provides investors with an alternative way to access the income and coupon payments of Treasury securities. STRIPS bonds are created by separating a Treasury security’s coupon and principal components and trading them as individual zero-coupon securities. This process allows investors to purchase and trade the coupon or principal components separately, providing greater flexibility in managing their investment portfolios.

<div>
    <center>
        <img src="figs/Fig-STRIPS-Schematic.png" width="900"/>
    </center>
</div>

For example, the 5-year Treasury note with annual coupon payments of $C$ USD and a face (par) value of $V_{P}$ (USD) shown above can be stripped into six separate zero-coupon securities, i.e., five zero-coupon bonds, each with face values of $C$ and maturity of $T$= 1,2,3,4 and 5 years, and a six security with face  (par) value of $V_{P}$ USD with a duration of $T$ = 5 years. In the general case, a treasury note or bond with $N=\lambda{T}$ coupon payments, where $T$ denotes the maturity in years, and $\lambda$ represents the number of coupon payments per year, can be stripped into $N+1$ separate zero-coupon securities.

Beyond thier immediate value as investment tools, STRIPS bonds are interesting as they provide another look at the [Term Structure of Interest Rates](https://www.investopedia.com/terms/t/termstructure.asp#:~:text=Essentially%2C%20term%20structure%20of%20interest,current%20state%20of%20an%20economy), i.e., how the change in the [short rates](https://en.wikipedia.org/wiki/Short-rate_model) influences the price and yeild the bond.

## Learning Objectives
In this example, we will create a model for a 30-year treasury bond with semi-annual coupon payments. Then, we will strip the coupon payments and face value to produce 61 zero-coupon STRIPS bonds. We will then show the bootstrapping method, an alternative approach to estimating the short rates from the price of the STRIPS zero-coupon products.

* __Objective 1__: Our first objective is to build a model for a 30-year treasury bond and strip the bond's coupon payments and face value. To make our calculations realistic, we will use coupon and market interest values from recent 30-year bond quotations. We will encounter an interesting challenge when we strip the 30-year bond, which is how to price the zero-coupon STRIP products.
* __Objective 2__: Our second objective is to demonstrate the bootstrapping method to estimate the short rates from the price of STRIP zero-coupon products.

## Setup
In the following code block, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our exercise:

* The [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package provides routines for univariate and multivariate optimization in [Julia](https://julialang.org). We use these routines to estimate the Yield to Maturity (YTM) of zero-coupon bonds generated from a parent treasury note or bond by stripping.
* The [LinearAlgebra.jl](https://github.com/JuliaLang/LinearAlgebra.jl) package provides routines for linear algebra operations. We use these routines to frame and solve (by computing a matrix inverse) the bootstrapping calculation to estimate the short rates from the price of zero-coupon bonds of different maturity.
* The [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package provides routines to display tabular data. We use these routines to display various calculation results throughout our exercises.

### Types
* The `MyUSTreasuryCouponSecurityModel` is [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) that holds important information such as the par value $V_{P}$, duration $T$, market interest rate $\bar{r}$, coupon rate $c$, and the number of coupon payments per year $\lambda$ for treasury notes and bonds. To create a `MyUSTreasuryCouponSecurityModel` instance, you can use the `build` method explained below.
* On the other hand, `MyUSTreasuryZeroCouponBondModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) that models a zero-coupon treasury bond. It holds the par value $V_{P}$, duration $T$, and interest rate $\bar{r}$ for a zero-coupon bond. You can create instances of `MyUSTreasuryZeroCouponBondModel` from a parent `MyUSTreasuryCouponSecurityModel` instance by using the `strip(...)` function explained below.
* Lastly, we export the `DiscreteCompoundingModel` and `ContinuousCompoundingModel` [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types), which indicate which compounding model we want to use. These types have no data associated with them.

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

`build(model::Type{MyUSTreasuryCouponSecurityModel}, data::NamedTuple) -> MyUSTreasuryCouponSecurityModel` 
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, etc.) and returns an instance of the `MyUSTreasuryCouponSecurityModel` [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types).

`strip(model::MyUSTreasuryCouponSecurityModel) -> Dict{Int, MyUSTreasuryZeroCouponBondModel}`
> This function takes a `MyUSTreasuryCouponSecurityModel` instance, i.e., a model of treasury security such as a note or bond with coupon payments that we want to strip and strips these payments from the parent security. The `strip(...)` function returns a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) holding `MyUSTreasuryZeroCouponBondModel` instances created from the parent security, where the keys of the dictionary correspond to the temporal index of the created security. 

`YTM(model::MyUSTreasuryCouponSecurityModel, compounding::T; rₒ::Float64 = 0.01) where T <: AbstractCompoundingModel` 
> This function computes the Yield to Maturity (YTM) given a `MyUSTreasuryCouponSecurityModel` instance, a compounding model, and an initial guess for the interest rate (default value of 1\%). This function computes the Internal Rate of Return (IRR) using the [Optim.jl](https://docs.sciml.ai/Optimization/stable/optimization_packages/optim/) package.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3`


## Objective 1: STRIP a 30-year treasury bond

In [2]:
# Set parameters for the original 30-year bond
Vₚ,T,r,c,λ = 1000.0,30.0,0.03832,0.03625,2

# Build a discrete compounding model -
discrete_compounding = DiscreteCompoundingModel();

# Build a 30-year model, and compute the price using the short-hand syntax
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r, coupon = c, λ = λ
        )) |> discrete_compounding;

# show -
println("The 30-year bond with face value Vₚ = $(Vₚ) USD is priced at $(round(model.price, sigdigits=5)) USD")

The 30-year bond with face value Vₚ = 1000.0 USD is priced at 963.28 USD


### Build the zero coupon STRIPS from the parent 30-year bond
We generate a dictionary of zero-coupon bond models for a `MyUSTreasuryCouponSecurityModel`, using the `strip`. The resulting `strips` variable stores the generated bonds in order of duration, with their indexes serving as keys and the corresponding `MyUSTreasuryZeroCouponBondModel` instances as values. 

In [3]:
strips = strip(model);

#### Challenge: How do we price the zero-coupon STRIPS?
STRIPS are often sold at discount compared to their face value because no coupon payments are made to holders. However, it’s important to note that the discount is determined by the seller in the secondary treasury market. To better understand this, let’s imagine a pricing scheme where we charge a varying percentage of the par value: 

$$V_{B} = \alpha\cdot{V}_{P}$$ 

Let $\alpha = (0.95)^T$ where T represents the maturity of the generated zero coupon bond. With the price, par value, and duration in hand, we can calculate the effective market interest $\bar{r}$ for each zero coupon security as a yield to maturity (YTM) calculation. 

#### Implementation
First, we save the count of the zero-coupon bonds generated by the `strip(…)` function in the `number_of_zc_bonds` variable. Then, we iterate through the indexes of the zero-coupon bonds, retrieving the individual bond models from the `strips` dictionary using the bracket notation `strips[i]`, where `i` represents the bond index. Next, we access the duration (the `T` field of `MyUSTreasuryZeroCouponBondModel`) and the par value (the `par` field of `MyUSTreasuryZeroCouponBondModel`) of a bond, which we use to compute the discount parameter $\alpha$ and the price of that bond. Finally, we compute (and set) the effective market rate of interest $\bar{r}$ (the `rate` field of `MyUSTreasuryZeroCouponBondModel`) by calling the `YTM(…)` function.

In [4]:
number_of_zc_bonds = length(strips)
for i ∈ 1:number_of_zc_bonds
    
    # get the model, access the data, compute α, and then the  price
    zc_model = strips[i];
    T = zc_model.T
    Vₚ = zc_model.par;
    
    # compute the discount
    α = 0.95^(T)
    
    # set the price = α*Vₚ
    zc_model.price = α*Vₚ
    
    # do a YTM calculation to compute the market rate of interest
    zc_model.rate = YTM(zc_model, discrete_compounding)
end

#### Visualize the first $N$ STRIPS products
To view a specified number of STRIPS zero-coupon bonds, we assign the desired number to the variable `number_of_strips_to_view`. Then, we loop through each bond index and access the respective models from the `strips` dictionary using the bracket notation `strips[i]`, where `i` represents the bond index. Finally, using the model, we add data to the `strips_table_data_array` and display the resulting table by calling the `pretty_table(…)` function.

In [5]:
number_of_strips_to_view = 10
strips_table_data_array = Array{Any,2}(undef, number_of_strips_to_view, 6);
strips_table_header_array = (["", "Maturity","Face","Price","Effective rate r̄", "Discount"],["","YR","USD","USD","%", "price/face"]);

for i ∈ 1:number_of_strips_to_view

    zc_model = strips[i];
    strips_table_data_array[i,1] = i
    strips_table_data_array[i,2] = zc_model.T
    strips_table_data_array[i,3] = zc_model.par
    strips_table_data_array[i,4] = zc_model.price
    strips_table_data_array[i,5] = zc_model.rate  
    strips_table_data_array[i,6] = (zc_model.price/zc_model.par)
end
pretty_table(strips_table_data_array, header=strips_table_header_array, tf = tf_html_default)

,Maturity,Face,Price,Effective rate r̄,Discount
,YR,USD,USD,%,price/face
1,0.5,18.125,17.6661,0.0526316,0.974679
2,1.0,18.125,17.2188,0.0526316,0.95
3,1.5,18.125,16.7828,0.0526316,0.925945
4,2.0,18.125,16.3578,0.0526316,0.9025
5,2.5,18.125,15.9436,0.0526316,0.879648
6,3.0,18.125,15.5399,0.0526316,0.857375
7,3.5,18.125,15.1464,0.0526316,0.835666
8,4.0,18.125,14.7629,0.0526316,0.814506
9,4.5,18.125,14.3891,0.0526316,0.793882


## Objective 2: Using bootstrapping to compute the short rates from STRIPS prices
We can estimate the _short rates_, which represent the market rate of interest between periods $j\rightarrow{j+1}$ and are denoted by $r_{j+1,j}$, by analyzing the prices of the various STRIPS zero coupon products based on their maturity. Using a discrete discounting model, the short rates are calculated according.:

$$
\begin{eqnarray}
\frac{V_{P,1}}{V_{B,1}} & = & \left(1+r_{1,0}\right) \\
\frac{V_{P,2}}{V_{B,2}} & = & \left(1+r_{2,1}\right)\cdot\left(1+r_{1,0}\right) \\
\vdots & = & \vdots \\
\frac{V_{P,N}}{V_{B,N}} & = & \prod_{i=1}^{N}\left(1+r_{i,i-1}\right) \\
\end{eqnarray}
$$

where $V_{P,i}$ and $V_{B,i}$ denote the face (par) value and price of the $i^{th}$ zero-coupon bond (both of which are known). Thus, we can solve for $r_{1,0}$, then insert that into the following expression to solve for $r_{2,1}$, and so on. Systematically, we can solve for the log-transformed short rates as a system of linear algebraic equations (LAEs) of the from:

$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

where $x_{i} = \log\left(1+r_{i,i-1}\right)$, $b_{i} = \log\left(V_{P,i}/V_{B,i}\right)$ and $\mathbf{A}$ is a lower-triangular matrix of `1`'s. We solve for the log-transformed short rates by computing the inverse of the matrix $\mathbf{A}$:

$$
\mathbf{x} = \mathbf{A}^{-1}\mathbf{b}
$$

and then transform these back to linear coordinates for each period:

$$
r_{i,i-1} = 10^{x_{i}} - 1
$$

#### Example: Setup of LAEs for $N=3$
Consider the case where $N = 3$ and we are using discrete compounding. In this case, after log-transforming the discounded price expressions, we have a $3\times{3}$ system of linear algebraic equations where each row represents a STRIPS zero-coupon bond:

$$
\begin{eqnarray}
\log\left(1+r_{1,0}\right) & = & \log\left(\frac{V_{P,1}}{V_{B,1}}\right)\\
\log\left(1+r_{1,0}\right)+\log\left(1+r_{2,1}\right) & = & \log\left(\frac{V_{P,2}}{V_{B,2}}\right)\\
\log\left(1+r_{1,0}\right)+\log\left(1+r_{2,1}\right) + \log\left(1+r_{3,2}\right)& = & \log\left(\frac{V_{P,3}}{V_{B,3}}\right)\\
\end{eqnarray}
$$

This system of equations can be re-arranged as a system of linear algebraic equations of the form:

$$
\begin{equation}
\begin{pmatrix}
1 & 0 & 0 \\
1 & 1 & 0 \\
1 & 1 & 1 \\
\end{pmatrix}
\begin{pmatrix}
\log\left(1+r_{1,0}\right) \\
\log\left(1+r_{2,1}\right) \\
\log\left(1+r_{3,2}\right)
\end{pmatrix} = 
\begin{pmatrix}
\log\left(V_{P,1}/V_{B,1}\right) \\
\log\left(V_{P,2}/V_{B,2}\right) \\
\log\left(V_{P,3}/V_{B,3}\right)
\end{pmatrix}
\end{equation}
$$

which is of the form $\mathbf{A}\mathbf{x} = \mathbf{b}$ where $x_{i} = \log\left(1+r_{i,i-1}\right)$, $b_{i} = \log\left(V_{P,i}/V_{B,i}\right)$.

### Compute short rates using boostrapping
In the code block below, we initialize and populate the $\mathbf{A}$ matrix, the right-hand size vector $\mathbf{b}$ and compute the matrix inverse using the [inv(...) function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Base.inv-Tuple{AbstractMatrix}) provided by the [LinearAlgebra.jl](https://github.com/JuliaLang/LinearAlgebra.jl) package. Finally, we transform the estimated short rates back to linear coordinates and add them to the `short_rate_array` array. 

In [6]:
short_rate_array = Array{Float64,1}(undef, number_of_strips_to_view);
b = Array{Float64,1}(undef, number_of_strips_to_view)
A = Array{Float64,2}(undef, number_of_strips_to_view, number_of_strips_to_view) |> (x-> fill!(x,0.0))


for i ∈ 1:number_of_strips_to_view
    for j ∈ 1:number_of_strips_to_view
        if (i ≥ j)
            A[i,j] = 1.0
        end
    end
end


for i ∈ 1:number_of_strips_to_view
    zc_model = strips[i];
    Vₚ = zc_model.par
    Vᵦ = zc_model.price
    b[i] = log10(Vₚ/Vᵦ);
end

# invert A, compute log transformed shorts
x̂ = inv(A)*b;

# convert log transformed shorts back to linear coordinates
for i ∈ 1:number_of_strips_to_view
    short_rate_array[i] = 10^(x̂[i]) - 1
end

### Visualize the estimated short rates
We visualized the short rates and compared the discount factors computed using the short rates to the discount factor calculated using the effective market rate of interest using the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. The `short_rate_data_table` array for `number_of_strips_to_view` bonds was populated by looping through each bond index and accessing the respective models from the `strips` dictionary using the bracket notation `strips[i]`, where `i` represents the bond index. The effective discount factor:

$$
\mathcal{D}_{T,0} = (1+\bar{r})^{T}
$$ 

and the multiperiod discrete discount factor:

$$
\mathcal{D}_{T,0} = \left[\prod_{j=0}^{T-1}\left(1+r_{j+1,j}\right)\right]
$$

were computed for each bond. Consistent with theory, the two formulations of the discount factor were equal:

In [19]:
short_rate_data_table = Array{Any,2}(undef, number_of_strips_to_view, 6)
short_rate_header_table = (
    ["index i","Period Start","Period End","Short rate","Discount short","Discount effective"], 
    ["","YR","YR","Decimal","dimensionless","dimensionless"]);
Tₒ = 0.0;
for i ∈ 1:number_of_strips_to_view
    
    zc_model = strips[i];
    T = zc_model.T
    r̄ = zc_model.rate;
    
    short_rate_data_table[i,1] = i
    short_rate_data_table[i,2] = Tₒ
    short_rate_data_table[i,3] = T;
    short_rate_data_table[i,4] = short_rate_array[i]
    
    # tmp -
    tmp = Array{Float64,1}();
    for j ∈ 1:i
        value = (1+short_rate_array[j])
        push!(tmp, value)
    end
    multiperiod = prod(tmp);
    effective = (1+r̄)^T
    short_rate_data_table[i,5] = multiperiod
    short_rate_data_table[i,6] = effective
    
    Tₒ = zc_model.T
end
pretty_table(short_rate_data_table, header=short_rate_header_table, tf = tf_html_default)

index i,Period Start,Period End,Short rate,Discount short,Discount effective
,YR,YR,Decimal,dimensionless,dimensionless
1,0.0,0.5,0.0259784,1.02598,1.02598
2,0.5,1.0,0.0259784,1.05263,1.05263
3,1.0,1.5,0.0259784,1.07998,1.07998
4,1.5,2.0,0.0259784,1.10803,1.10803
5,2.0,2.5,0.0259784,1.13682,1.13682
6,2.5,3.0,0.0259784,1.16635,1.16635
7,3.0,3.5,0.0259784,1.19665,1.19665
8,3.5,4.0,0.0259784,1.22774,1.22774
9,4.0,4.5,0.0259784,1.25963,1.25963
